# The Eikonal Equation with scikit-fmm and DeepXDE

In [ ]:
import skfmm
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde

## Problem definition and scikit-fmm

In [ ]:
# Unit square computational domain
xmin, xmax = 0.0, 1.0
ymin, ymax = 0.0, 1.0
nx, ny = 101, 101
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

In [ ]:
# Create mesh
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)
X, Y = np.meshgrid(x, y)

In [ ]:
# Define boundary as zero contour of surface phi
xs, ys = 0.2, 0.8 # source point
phi = np.ones_like(X)
phi[int(np.floor(ys*ny)), int(np.floor(xs*nx))] = -1
plt.contour(X, Y, phi,[0], linewidths=(3), colors='black')
plt.title('Boundary')
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
# Compute the distance from the boundary
d = skfmm.distance(phi, dx=dx)
plt.contour(X, Y, d, 15)
plt.title('Signed distance from boundary')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
# Define linear/constant velocity field
constant = True
v0 = 1.0
if constant:
    dvdx, dvdy = 0.0, 0.0
else:
    dvdx, dvdy = 0.5, 0.5
def v(x, y):
    return v0 + dvdx*x + dvdy*y
V = v(X, Y)
if not constant:
    plt.contour(X, Y, V, 15)
    plt.title('Velocity field')
    plt.colorbar()
    plt.xlabel('x')
    plt.ylabel('y');

In [ ]:
# Compute travel times using fast marching method (FMM)
U_fm = skfmm.travel_time(phi, V, dx=dx)
plt.contour(X, Y, U_fm, 25)
plt.title('Travel time from boundary')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y');

## PINN solution with DeepXDE

In [ ]:
# Check that velocity field is constant
assert constant, 'The DeepXDE code only works with a constant velocity field'

In [ ]:
# Reference solution
def u_ref(x):
    xt = x[:,0]
    yt = x[:,1]
    return np.sqrt((xt-0.2)**2 + (yt-0.8)**2).reshape(-1, 1)

In [ ]:
# Choose points to use for model training
n_obs = 30
random_indices = np.random.choice(X.size, size=n_obs, replace=False)
x_obs = np.column_stack((
    X.flatten()[random_indices],
    Y.flatten()[random_indices]
))
u_obs = U_fm.flatten()[random_indices].reshape(-1, 1)

In [ ]:
# Define computational domain
geom = dde.geometry.Rectangle([xmin,ymin], [xmax,ymax])

In [ ]:
# 2D Eikonal equation
def eikonal(x, y):
    """
    |du/dx| = 1/v(x)
    """  
    du_x = dde.grad.jacobian(y, x, i=0, j=0)
    du_y = dde.grad.jacobian(y, x, i=0, j=1)

    xt = x[:,0]
    yt = x[:,1]
    
    return du_x**2 + du_y**2 - v(xt, yt)**-2
    # return du_x*0.0

In [ ]:
# Boundary condition for source point
bc = dde.icbc.PointSetBC([0.2, 0.8], 0.0)

In [ ]:
# Training data
# obs = dde.icbc.PointSetBC(x_obs, u_obs)
obs = dde.icbc.PointSetBC(x_obs, u_ref(x_obs))

In [ ]:
# Combined DeepXDE data object
data = dde.data.PDE(
    geom,
    eikonal,
    # [bc, obs],
    [obs], # works better without source point separately specified
    num_domain=100,
    num_boundary=0,
    anchors=x_obs,
    solution=u_ref
)

In [ ]:
# Neural network
net = dde.nn.FNN([2] + [10] * 3 + [1], "tanh", "Glorot uniform")
model = dde.Model(data, net)

In [ ]:
# Compile DeepXDE model
model.compile("adam", lr=0.001)

In [ ]:
# Train model
losshistory, train_state = model.train(iterations=30000)

In [ ]:
# Plot results
dde.saveplot(losshistory, train_state, issave=False, isplot=True)

In [ ]:
# Make predictions over whole domain
U_pinn = model.predict(np.column_stack((X.flatten(), Y.flatten()))).reshape(-1,101)

In [ ]:
# Plot predicted travel times from PINN
plt.contour(X, Y, U_pinn, 25)
plt.title('Travel time predicted by PINN')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y');